In [ ]:
# Instalar librerías, en caso de que no las tengan instaladas.
!pip install --upgrade pip
!pip install -r requirements.txt

In [ ]:
import fasttext
import numpy
import gensim
import logging
import gensim.downloader as api

In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
wv = api.load('word2vec-google-news-300')

In [ ]:
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

In [ ]:
vec_king = wv['king']
print(vec_king)


In [ ]:
vec_male = wv['male']
print(vec_male)

In [ ]:
vec_queen = wv['queen']
print(vec_queen)


In [ ]:
difference = vec_king - vec_male
print(difference)

In [ ]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

In [ ]:
pairs = [
    ('king', 'queen'),   # a minivan is a kind of car
    ('king', 'male'),   # still a wheeled vehicle
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

In [ ]:
print(wv.most_similar(positive=['car', 'minivan'], topn=5))

El modelo no es capaz de inferir un vector para una palabra desconocida

In [ ]:
try:
    vec_cameroon = wv['cameroon']
except KeyError:
    print("The word 'cameroon' does not appear in this model")

Entrenar un modelo propio

In [ ]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = open('corpus/facundo.txt')
        for line in corpus_path:
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [ ]:
sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences)

In [ ]:
vec_Facundo = model.wv['facundo']
print(vec_Facundo)

In [ ]:
print(model.wv.most_similar(positive=['facundo'], topn=5))

In [ ]:




# Train Latent Semantic Indexing with 200D vectors.
lsi = models.LsiModel(corpus, num_topics=200)

# Convert another corpus to the LSI space and index it.
index = similarities.MatrixSimilarity(lsi[another_corpus])

# Compute similarity of a query vs indexed documents.
sims = index[query]